In [25]:
using System.IO;

In [26]:
public struct Point
{
    public Point(int _x, int _y)
    {
        x = _x;
        y = _y;
    }

    public int x;
    public int y;
}

public struct Rect
{
    public Point UpperLeft;
    public Point LowerRight;
}

public class Board
{
    public List<(int number, Rect rect)> Numbers = new List<(int number, Rect rect)>();
    public List<Rect> Symbols = new List<Rect>();
}

public Board ParseBoard(string[] lines)
{
    var board = new Board();

    for (var y = 0; y < lines.Count(); y++)
    {
        var characters = lines[y].ToCharArray();
        var number = "";
        var rect = new Rect();
        for (var x = 0; x < characters.Count(); x++)
        {
            if (Char.IsDigit(characters[x]))
            {
                if (number == "") // we're at the beginning of a number
                {
                    rect.UpperLeft = new Point(x-1, y-1);
                }

                number += characters[x];

            }
            
            if (characters[x] == '.' || x+1 == characters.Count())
            {
                if (number != "") // we've hit the end of a number
                {
                    rect.LowerRight = new Point(x, y+1);
                    
                    board.Numbers.Add((int.Parse(number), rect));
                    number = "";
                    rect = new Rect();
                }

                continue;
            }

            if (characters[x] != '.' && !Char.IsDigit(characters[x])) 
            {
                if (number != "") // we've hit the end of a number
                {
                    rect.LowerRight = new Point(x, y+1);
                    
                    board.Numbers.Add((int.Parse(number), rect));
                    number = "";
                    rect = new Rect();
                }
                
                var point = new Point(x, y);
                board.Symbols.Add(new Rect()
                {
                    UpperLeft = point,
                    LowerRight = point
                });

                continue;
            }
        }
    }

    return board;
}

public bool RectsOverlap (Rect rect1, Rect rect2)
{
    if (rect1.LowerRight.x < rect2.UpperLeft.x || rect2.LowerRight.x < rect1.UpperLeft.x)
    {
        return false;
    }

    if (rect1.LowerRight.y < rect2.UpperLeft.y || rect2.LowerRight.y < rect1.UpperLeft.y)
    {
        return false;
    }

    return true;
}

In [27]:
// Part One

var input = File.ReadAllLines("./input.txt");

var board = ParseBoard(input);

board
    .Numbers
    .Where(n => 
    {
        foreach (var symbol in board.Symbols)
        {
            if (RectsOverlap(n.rect, symbol))
                return true;
        }

        return false;
    })
    .Select(n => n.number)
    .Sum()

530495

In [28]:
// Part Two

board
    .Symbols
    .Select(s => 
    {
        var nums = new List<int>();

        foreach (var number in board.Numbers)
        {
            if (RectsOverlap(number.rect, s))
                nums.Add(number.number);
        }

        return nums;
    })
    .Where(nums => nums.Count() == 2)
    .Select(nums => nums.Aggregate((a,b) => a*b))
    .Sum()

80253814